In [1]:
from discourse_tree_utils import *
from collections import defaultdict
from string import digits
import glob, random, json
import pandas as pd
pd.set_option('display.max_colwidth', -1)
ISO='ISO-8859-1'

In [2]:
path = '/home/ffajri/Data/ES_DTB/ALL/*/*.rs3'
files = glob.glob(path)
random.shuffle(files)

In [3]:
TRAIN = files[:200]
DEV = files [200:230]
TEST = files[230:]

In [24]:
def get_text(graph, span):
    start_index= int(span[3])
    end_index = int(span[4])+1
    return ' '.join([graph.node[a]['label'] for a in graph.tokens[start_index:end_index]])

def subprocess(graph, spans):
    edu1 = get_text(graph, spans[0])
    nuc1 = spans[0][1].translate(None, digits)
    rel = spans[0][2]
    
    if len(spans) == 2:
        edu2 = get_text(graph, spans[1])
    else:
        edu2 = []
        for span in spans[1:]:
            edu2.append(get_text(graph, span))
        edu2 = ' '.join(edu2)
    nuc2 = spans[-1][1].translate(None, digits)
    
    return edu1, edu2, nuc1+nuc2, rel
    
def process(graph, spans):
    span_dict = defaultdict(list)
    for span in spans:
        span_dict[span[0]].append(span)
    for span in spans:
        temp = sorted(span_dict[span[0]], key=lambda x: x[3])
        span_dict[span[0]] = temp
        
    edus1 = []; edus2 = []; nucs = []; rels = []
    for key in span_dict.keys():
        edu1, edu2, nuc, rel = subprocess(graph, span_dict[key])
        if edu1=='' or edu2=='':
            continue
        edus1.append(edu1)
        edus2.append(edu2)
        nucs.append(nuc)
        rels.append(rel)
    
    df = pd.DataFrame()
    df['edu1']=edus1; df['edu2']=edus2; df['nuclear']=nucs; df['relation']=rels
    
    edus = []
    for edu in get_edus(graph):
        edus.append(graph.node[edu]['rst:text'])
    
    return df, edus
    
def write_segment(fname, array):
    final_data={}
    for idx, data in enumerate(array):
        final_data[idx]=data
    json.dump(final_data, open(fname, 'w'))

def compute_and_save(TARGET, fname):
    final_df = pd.DataFrame(columns=['edu1', 'edu2', 'nuclear', 'relation'])
    all_edus = []
    for file in TARGET:
        #print (file)
        graph = RSTGraph(file, iso=ISO)
        spans = get_rst_spans(graph)
        df, edus = process(graph, spans)
        final_df = final_df.append(df, ignore_index=True)
        all_edus.append(edus)
    
    #save rst nuclear relation
    final_df.to_csv('/home/ffajri/Workspace/WhatDiscourse/segment/data/data_es/'+fname+'.csv', index=False,  encoding=ISO)
    #save segmentation
    write_segment('/home/ffajri/Workspace/WhatDiscourse/segment/data/data_es/'+fname+'_edu.json', all_edus)

In [26]:
#NUCLEARITY and RELATION PREDICTION and SEGMENTATION

compute_and_save(TRAIN, 'train')
compute_and_save(DEV, 'dev')
compute_and_save(TEST, 'test')